In [5]:
print("ok")

ok


In [6]:
import os
os.chdir("../")

In [7]:
%pwd

'd:\\Projects AI\\End-to-End-Medical-Chatbot-Generative-AI'

In [8]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [9]:
# Extract Data From the PDF File
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents

In [10]:
extracted_data=load_pdf_file(data='Data/')

In [11]:
# extracted_data

In [12]:
# Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks


In [13]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 2697


In [14]:
# text_chunks

In [15]:
from langchain.embeddings import HuggingFaceEmbeddings

In [16]:
# Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [17]:
embeddings = download_hugging_face_embeddings()

C:\Users\richa\AppData\Local\Temp\ipykernel_18136\565510569.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\richa\anaconda3\envs\medibot\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\richa\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by

In [19]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
# query_result

[-0.03447727486491203,
 0.03102312609553337,
 0.006734980270266533,
 0.026108933612704277,
 -0.03936205804347992,
 -0.16030246019363403,
 0.06692394614219666,
 -0.006441438104957342,
 -0.047450482845306396,
 0.014758863486349583,
 0.07087534666061401,
 0.05552757531404495,
 0.019193356856703758,
 -0.02625126577913761,
 -0.01010954286903143,
 -0.026940442621707916,
 0.022307462990283966,
 -0.02222665585577488,
 -0.14969263970851898,
 -0.017493024468421936,
 0.007676282897591591,
 0.054352231323719025,
 0.0032544038258492947,
 0.03172588348388672,
 -0.08462139964103699,
 -0.029405992478132248,
 0.051595550030469894,
 0.048124078661203384,
 -0.003314835485070944,
 -0.05827915295958519,
 0.04196925833821297,
 0.022210702300071716,
 0.1281888633966446,
 -0.022338951006531715,
 -0.011656239628791809,
 0.06292837113142014,
 -0.03287634998559952,
 -0.09122604131698608,
 -0.03117534890770912,
 0.052699536085128784,
 0.04703483358025551,
 -0.08420310169458389,
 -0.030056182295084,
 -0.0207448396

In [21]:
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-mouhzy6.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [27]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [29]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


In [30]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)


In [32]:
docsearch

In [34]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [35]:
retrieved_docs = retriever.invoke("What is Colon?")

In [36]:
retrieved_docs

[Document(id='3018e46f-ac79-42e0-8f2e-e00e649749ff', metadata={'author': 'BOOKSMEDICOS.ORG', 'creationdate': '2015-11-10T20:01:27-07:00', 'creator': 'Nitro Pro 9', 'keywords': 'BOOKSMEDICOS.ORG', 'moddate': '2015-11-10T20:04:18-07:00', 'page': 245.0, 'page_label': '246', 'producer': 'PyPDF', 'source': 'Data\\Gastro_libro.pdf', 'subject': 'BOOKSMEDICOS.ORG', 'title': 'Atlas de imágenes en gastroenterología: correlación radiología-endoscopia', 'total_pages': 500.0}, page_content='232   COLON Y RECTO\nD\n E\n F\n FIGURA 1.4.4 (Continuaci ó n)           C á ncer col ó nico como causa de obstrucci ó n intestinal.  La imagen de la TC coronal reformateada \ncurvada  (D)  obtenida en un segundo paciente muestra un tumor obstructivo en el colon transverso proximal  (flecha) . La dilataci ó n \ndel colon proximal relacionada con el intestino delgado posiblemente se deba a la competencia de la v á lvula ileocecal. La imagen'),
 Document(id='ea9e76de-1957-4259-b9bc-50be76edb0bc', metadata={'author

In [37]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.4, max_tokens=500)


KeyboardInterrupt: 